# Example NTV

In [1]:
from ntv import NtvSingle, NtvList, NtvSet, Ntv

## All JSON data is JSON-NTV data
Codification : 
- first letter: "v" (NtvSingle), "l" (NtvList), "s" (NtvSet)
- additional letters: 'N' (named), 'T' (typed)

In [2]:
liste = [None, 1, [1,2], {'key': 'value'}, {'key1': 'val1', 'key2': 'val2'}, 
         {'example': [21, [1,2], {'key1': 3, 'key2': 4}]}]
for js in liste:
    ntv = Ntv.obj(js)
    print('{:<50} {} {}'.format(str(ntv), 'codification : ', repr(ntv)))

null                                               codification :  "v"
1                                                  codification :  "v"
[1, 2]                                             codification :  {"l": ["v", "v"]}
{"key": "value"}                                   codification :  "vN"
{"key1": "val1", "key2": "val2"}                   codification :  {"s": ["vN", "vN"]}
{"example": [21, [1, 2], {"key1": 3, "key2": 4}]}  codification :  {"lN": ["v", {"l": ["v", "v"]}, {"s": ["vN", "vN"]}]}


## NTV data is typed
- default type is 'json'


In [3]:
liste = [[{':json' : 21}, 21]]
for js in liste:
    print(Ntv.obj(js[0]) == Ntv.obj(js[1]))

False


- types can be predefined

In [4]:
cities1 = Ntv.obj({'cities::point': [{'paris': [4.1, 40.5]}, [4.5, 41.2]]})
cities2 = Ntv.obj({'cities::': [{'paris': [4.1, 40.5]}, [4.5, 41.2]]})
cities3 = Ntv.obj([{'paris': [4.1, 40.5]}, [4.5, 41.2]])
cities4 = Ntv.obj({'cities:': [{'paris': [4.1, 40.5]}, [4.5, 41.2]]})
print(cities1.to_obj(encode_format='obj', simpleval=True))
print(cities2.to_obj(encode_format='obj', simpleval=True))
print(cities3.to_obj(encode_format='obj', simpleval=True))
print('\n', repr(cities1), '\n', repr(cities2), '\n', repr(cities3), '\n', repr(cities4))

{'::point': [<shapely.geometry.point.Point object at 0x00000170C0E7DDB0>, <shapely.geometry.point.Point object at 0x00000170C0E7DD50>]}
[[4.1, 40.5], [4.5, 41.2]]
[[4.1, 40.5], [4.5, 41.2]]

 {"lNT": ["vNT", "vT"]} 
 {"lN": [{"lN": ["v", "v"]}, {"l": ["v", "v"]}]} 
 {"l": [{"lN": ["v", "v"]}, {"l": ["v", "v"]}]} 
 "vN"


In [5]:
print(cities1.to_obj(simpleval=True))

{'::point': [[4.1, 40.5], [4.5, 41.2]]}


In [6]:
cities1

{"lNT": ["vNT", "vT"]}

In [7]:
print(cities1)
list_cities = list(cities1)
print(list_cities[0])

{"cities::point": [{"paris": [4.1, 40.5]}, [4.5, 41.2]]}
{"paris:point": [4.1, 40.5]}


In [8]:
json = Ntv.obj({'student': [{'name': ['john', 'eric', 'judith']}, 
                               {'age': [15, 54, 48]}, 
                               {'city': [{"paris": [4.1, 40.5]}, [4.5, 41.2]]}]})
print(json.to_obj(simpleval=True))

[['john', 'eric', 'judith'], [15, 54, 48], [[4.1, 40.5], [4.5, 41.2]]]


In [9]:
df = Ntv.obj({'student:tab': [{'name::string': ['john', 'eric', 'judith']}, 
                               {'age::number': [15, 54, 48]}, 
                               {'city::point': [{"paris": [4.1, 40.5]}, [4.5, 41.2], [2.5, 40.2]]}]})
df2 = Ntv.obj({'student': [{'name::string': ['john', 'eric', 'judith']}, 
                           {'age::number': [15, 54, 48]}, 
                           {'city::point': [{"paris": [4.1, 40.5]}, [4.5, 41.2], [2.5, 40.2]]}]})
print(repr(df), '\n', repr(df2), '\n')
print(df.to_obj(encode_format='tuple'))
fields = list(df2)
print(fields[1])
print(fields[1][2])

"vNT" 
 {"lN": [{"lNT": ["vT", "vT", "vT"]}, {"lNT": ["vT", "vT", "vT"]}, {"lNT": ["vNT", "vT", "vT"]}]} 

('student', 'tab', [{'name::string': ['john', 'eric', 'judith']}, {'age::number': [15, 54, 48]}, {'city::point': [{'paris': [4.1, 40.5]}, [4.5, 41.2], [2.5, 40.2]]}])
{"age::number": [15, 54, 48]}
{":number": 48}


In [10]:
lis = [[4.1, 40.5], [4.5, 41.2], [2.5, 40.2]]
#lis2 = {':':[[4.1, 40.5], [4.5, 41.2], [2.5, 40.2]]}
nlis =Ntv.obj(lis)

nlis.set_name('cities')
nlis[0].set_name('paris')
print(nlis)
print(repr(nlis))

nlis.set_type('point')
print(nlis)
print(repr(nlis))

{"cities": [{"paris": [4.1, 40.5]}, [4.5, 41.2], [2.5, 40.2]]}
{"lN": [{"lN": ["v", "v"]}, {"l": ["v", "v"]}, {"l": ["v", "v"]}]}
{"cities::point": [{"paris": [4.1, 40.5]}, [4.5, 41.2], [2.5, 40.2]]}
{"lNT": [{"lN": ["v", "v"]}, {"l": ["v", "v"]}, {"l": ["v", "v"]}]}
